In [ ]:
import pandas as pd
from collections import OrderedDict

1. 读取AMSA1 file and New_ini_Target文件（在A101中生成的文件）
2. 提取target value,存入AMSA1中的新列“New Target”，然后将AMSA1文件另存为AMSA2.


1. read the AMSA1 file and new_ini_Target file (which is generate from programme A101)
2. extract ini file target value, loading the target value in AMSA1, create new column named'new target', save AMSA1 file as AMSA2 

In [ ]:

amsa_doc = pd.read_excel('AMSA1.xlsx', sheet_name=None)

# reading ini file.
new_ini_doc = pd.read_excel('New_ini_Target.xlsx', sheet_name=None)


writer = pd.ExcelWriter('AMSA2.xlsx', engine='xlsxwriter')

# create the dic to save working sheet
ordered_sheets = OrderedDict()

# sweep all ini traget file sheet and contents
for sheet_name, ini_sheet in new_ini_doc.items():
    print(sheet_name)
    # if sheet name is not"94", then skip
    if sheet_name != '94':
        continue

    # get all the frequency value from sheet name 
    freq_values = ini_sheet['Freq'].unique()
    # print(freq_values)

    #获取相对应的value列的读值
    for freq in freq_values:
        value = ini_sheet.loc[ini_sheet['Freq'] == freq, 'Value'].values[0]  # Freq Amplitude Target in inifile value
        value2=ini_sheet.loc[(ini_sheet['Freq'] == freq) & (ini_sheet['Test']=='goldenphase'), 'Value'].values[0]  #Phase Target in inifile value


        #  change all the freq value to be int, prepare to do searching in AMSA1
        freq=int(freq)
        Phase=f'Phase_{freq}'
        # print(freq)
        # print(type(freq))
        # print(type(Phase))

        processed = False

        for sheet_name2, amsa_sheet in amsa_doc.items():
            if str(freq)==sheet_name2 and not processed:
                #  add new column, value is same as New_Target, but column title is frequncey value  
                amsa_sheet.loc[:, 'New_Target'] = value

                # each AMSA_sheet saved in indepent working sheet, using 'AMSA-{}' as working sheet name
                sheet_name3 = '{}'.format(freq)
                amsa_sheet.to_excel(writer, sheet_name=sheet_name3, index=False)
                ordered_sheets[sheet_name3] = amsa_sheet
                #marked sheet is processed 
                processed = False

        for sheet_name2, amsa_sheet in amsa_doc.items():

            # print(sheet_name2)

            # if Phase in sheet_name2 and not processed:
            if str(Phase)==sheet_name2:
                # print('phaseok')
            
                amsa_sheet.loc[:, 'Phase_New_Target'] = value2

               # each AMSA_sheet saved in indepent working sheet, using 'AMSA-{}' as working sheet name
               
                sheet_name4 = '{}'.format(Phase)
                amsa_sheet.to_excel(writer, sheet_name=sheet_name4, index=False)
                ordered_sheets[sheet_name4] = amsa_sheet
                #marked sheet is processed
                processed = True

        for sheet_name2, amsa_sheet in amsa_doc.items():
            if 'THD' in sheet_name2:
                amsa_sheet.to_excel(writer, sheet_name=sheet_name2, index=False)
                ordered_sheets[sheet_name2] = amsa_sheet
            else:
                continue

    

writer.close()
